# Case Study #2

In [71]:
len(X_train), len(X_test),X_train.shape, X_test.shape

(42000, 18000, (42000, 32, 32), (18000, 32, 32))

In [72]:
X_train = X_train.reshape(X_train.shape[0], 1024)
X_test = X_test.reshape(X_test.shape[0], 1024)
# Normalize inputs from 0-255 to 0-1

X_train = X_train/255
X_test = X_test/255

In [49]:
svh_x_train, svh_x_test, svh_y_train, svh_y_test = X_train, X_test, y_train, y_test

In [50]:
student_X_train, student_X_val, student_y_train, student_y_val = train_test_split(student_x_train, student_y_train, test_size=1/3, random_state=42)
cifar_X_train, cifar_X_val, cifar_y_train, cifar_y_val = train_test_split(cifar_x_train, cifar_y_train, test_size=1/3, random_state=42)
svh_X_train, svh_X_val, svh_y_train, svh_y_val = train_test_split(svh_x_train, svh_y_train, test_size=1/3, random_state=42)

In [86]:
from cuml.datasets.classification import make_classification
from cuml.model_selection import train_test_split
from cuml.ensemble import RandomForestClassifier as cuRF
from sklearn.metrics import accuracy_score

# synthetic dataset dimensions
n_samples = 1000
n_features = 10
n_classes = 2

# random forest depth and size
n_estimators = 25
max_depth = 10

# generate synthetic data [ binary classification task ]
X, y = make_classification ( n_classes = n_classes,
                             n_features = n_features,
                             n_samples = n_samples,
                             random_state = 0 )

X_train, X_test, y_train, y_test = train_test_split( X, y, random_state = 0 )

model = cuRF( max_depth = max_depth,
              n_estimators = n_estimators,
              random_state  = 0 )

trained_RF = model.fit ( X_train, y_train )

predictions = model.predict ( X_test )

cu_score = cuml.metrics.accuracy_score( y_test, predictions )
sk_score = accuracy_score( asnumpy( y_test ), asnumpy( predictions ) )

print( " cuml accuracy: ", cu_score )
print( " sklearn accuracy : ", sk_score )

# save
dump( trained_RF, 'RF.model')

# to reload the model uncomment the line below
loaded_model = load('RF.model')

../../thread/thread_load.cuh(36): warning: cuda.h: [jitify] File not found
../../thread/thread_store.cuh(36): warning: cuda.h: [jitify] File not found


/home/ahmad/anaconda3/envs/rapids-23.12/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


NameError: name 'asnumpy' is not defined

In [87]:
print( " cuml accuracy: ", cu_score )


 cuml accuracy:  0.9639999866485596


In [83]:
import time
from cuml.ensemble import RandomForestClassifier as cuRF
from cuml.metrics import accuracy_score
from sklearn.utils.extmath import cartesian
from tabulate import tabulate

def find_best_rf_parameters(X_train, y_train, X_val, y_val):
    # Define a function to reshape data for GPU
    def reshape_data(X):
        if len(X.shape) == 4:
            n_samples, img_height, img_width, n_channels = X.shape
            return X.reshape((n_samples, img_height * img_width * n_channels))
        elif len(X.shape) == 3:
            n_samples, img_height, img_width = X.shape
            return X.reshape((n_samples, img_height * img_width))
        else:
            return X

    X_train = reshape_data(X_train)
    X_val = reshape_data(X_val)

    rf_param_grid = {
        'n_estimators': [10, 50, 100],
        'split_criterion': ['entropy', 'gini'],
        'max_depth': [10, 50, 100],
        'max_features': ["sqrt", "log2"]
    }

    # Create a Cartesian product of parameter combinations
    rf_grid = ParameterGrid(rf_param_grid)

    data = []
    head = ['n_estimators', 'split_criterion', 'max_depth', 'max_features', 'score in validation set']

    start_time = time.time()
    for param in rf_grid:
        print(param)
        # n_estimators, split_criterion, max_depth, max_features = param
        # rf_model = RandomForestClassifier(n_estimators=int(n_estimators), split_criterion=split_criterion, max_depth=int(max_depth),
        #                                  max_features=max_features)
        rf_model = RandomForestClassifier()
        rf_model.fit(X_train, y_train)
        y_pred = rf_model.predict(X_val)
        score = accuracy_score(y_val, y_pred)
        list_entry = [param['n_estimators'],  param['split_criterion'], param['max_depth'], param['max_features'], score]
        data.insert(0, list_entry)
    
    # Find the best model
    best_model = max(data, key=lambda x: x[4])

    # Print the results in a table format
    print(tabulate(data, headers=head, tablefmt="pipe"))
    print("--- %s seconds ---" % (time.time() - start_time))
    print("The highest Accuracy {:.5f} is the model with n_estimators = {}, criterion = '{}', max_depth = {}, and max_features = '{}'"
          .format(best_model[4], best_model[0], best_model[1], best_model[2], best_model[3]))
    return best_model



In [ ]:
# Assuming you have loaded your datasets as student_X_train, student_X_val, student_y_train, student_y_val, 
# cifar_X_train, cifar_X_val, cifar_y_train, cifar_y_val, svh_X_train, svh_X_val, svh_y_train, svh_y_val
best_student_model = find_best_rf_parameters(student_X_train, student_y_train, student_X_val, student_y_val)

In [ ]:
best_cifar_model = find_best_rf_parameters(cifar_X_train, cifar_y_train, cifar_X_val, cifar_y_val)
best_svh_model = find_best_rf_parameters(svh_X_train, svh_y_train, svh_X_val, svh_y_val)

In [94]:
def find_best_rf_parameters(X_train, y_train, X_val, y_val):
        
    rf_param_grid = {
        'n_estimators': [10, 50, 100],
        'criterion': ['entropy', 'gini'],
        'max_depth': [10, 50, 100],
        'max_features': ["sqrt", "log2"]
    }
    rf_grid = ParameterGrid(rf_param_grid)
    data = []
    head = ['n_estimators', 'criterion', 'max_depth', 'max_features', 'score in validation set']

    start_time = time.time()
    for param in rf_grid:
        rf_model = RandomForestClassifier(**param, n_jobs = -1)
        rf_model.fit(X_train, y_train)
        score = rf_model.score(X_val, y_val)
        list_entry = [param['n_estimators'],  param['criterion'], param['max_depth'], param['max_features'], score]
        data.insert(0, list_entry)

    # Find the best model
    max_accuracy = max(entry[4] for entry in data)
    best_model = max(data, key=lambda x: x[4])

    # Print the results in a table format
    print(tabulate(data, headers=head, tablefmt="pipe"))
    print("--- %s seconds ---" % (time.time() - start_time))
    print("The highest Accuracy {:.5f} is the model with n_estimators = {}, criterion = '{}', max_depth = {}, and max_features = '{}'"
          .format(best_model[4], best_model[0], best_model[1], best_model[2], best_model[3]))
    return best_model

In [86]:
best_params_student = find_best_rf_parameters(student_X_train, student_y_train.values.flatten(), student_X_val, student_y_val.values.flatten())

|   n_estimators | criterion   |   max_depth | max_features   |   score in validation set |
|---------------:|:------------|------------:|:---------------|--------------------------:|
|            100 | gini        |         100 | log2           |                  0.769074 |
|             50 | gini        |         100 | log2           |                  0.772126 |
|             10 | gini        |         100 | log2           |                  0.741607 |
|            100 | gini        |         100 | sqrt           |                  0.769074 |
|             50 | gini        |         100 | sqrt           |                  0.770092 |
|             10 | gini        |         100 | sqrt           |                  0.766022 |
|            100 | gini        |          50 | log2           |                  0.772126 |
|             50 | gini        |          50 | log2           |                  0.765005 |
|             10 | gini        |          50 | log2           |                 

In [87]:
best_params_cifar = find_best_rf_parameters(cifar_X_train, cifar_y_train, cifar_X_val, cifar_y_val)

|   n_estimators | criterion   |   max_depth | max_features   |   score in validation set |
|---------------:|:------------|------------:|:---------------|--------------------------:|
|            100 | gini        |         100 | log2           |                  0.194396 |
|             50 | gini        |         100 | log2           |                  0.175976 |
|             10 | gini        |         100 | log2           |                  0.107818 |
|            100 | gini        |         100 | sqrt           |                  0.201896 |
|             50 | gini        |         100 | sqrt           |                  0.179756 |
|             10 | gini        |         100 | sqrt           |                  0.112198 |
|            100 | gini        |          50 | log2           |                  0.197276 |
|             50 | gini        |          50 | log2           |                  0.172677 |
|             10 | gini        |          50 | log2           |                 

In [89]:
svh_X_train.shape

(28000, 32, 32)

In [95]:
best_params_svh = find_best_rf_parameters(svh_X_train, svh_y_train, svh_X_val, svh_y_val)

KeyboardInterrupt: 

In [1]:
import cuml